In [48]:
using Distances, DataFrames, Gadfly, Distributions

include("TaxonomyExample.jl")
include("BlahutArimoto.jl")
include("InformationTheoryFunctions.jl")
include("ConversionFunctions.jl")
include("VisualizationFunctions.jl")
include("ThreeVariableBlahutArimoto.jl")

threevarBAiterations (generic function with 1 method)

In [49]:
#set up taxonomy example
include("TaxonomyExample.jl")
w_vec, w_strings, a_vec, a_strings, p_w, U = setuptaxonomy()


#pre-compute utilities, find maxima
U_pre, Umax = setuputilityarrays(a_vec,w_vec,U)

#visualize utility
#plt_utility = visualizeMatrix(U_pre, w_vec, a_vec, w_strings, a_strings, xlabel="World state w",
#                              ylabel="Action a", legendlabel="U(a,w)")

(
19x13 Array{Float64,2}:
 3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  3.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0
 2.2  2.2  2.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  2.2  2.2  2.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [56]:
β1 = 1 #inverse temperature for p(o)→p(o|w)
β2 = 1 #inverse temperature for p(a)→p(a|o)
β3 = 1 #inverse temperature for p(a)→p(a|o,w)

ε = 0.0000001 #convergence critetion for BAiterations
maxiter = 10000 #maximum number of BA iterations


#cardinality of percept
num_obs = length(a_vec)-1 #cardinality of percept is equal to action-space


#----- Initialization -----#

num_worldstates = length(w_vec)
num_acts = length(a_vec)

#initialize p(o|w)
#p_ogw_init = ones(num_obs, num_worldstates)  #uniform initialization
p_ogw_init = rand(num_obs, num_worldstates)  #random initialization
for j in 1:num_worldstates    
    p_ogw_init[:,j] = p_ogw_init[:,j] / sum(p_ogw_init[:,j])
end

#initialize p(a|o,w)
#p_agow_init = ones(num_acts, num_obs, num_worldstates) #uniform initialization
p_agow_init = rand(num_acts, num_obs, num_worldstates) #random initialization
for j in 1:num_worldstates
    for k in 1:num_obs
        p_agow_init[:,k,j] = p_agow_init[:,k,j] / sum(p_agow_init[:,k,j])
    end
end



#This function performs Blahut-Arimoto iterations for the three-variable general case
po, pa, pogw, pago, pagow, performance_df = threevarBAiterations(p_ogw_init, p_agow_init, β1, β2, β3, 
                                            U_pre, p_w, ε, maxiter, compute_performance=true, 
                                            performance_per_iteration=false, performance_as_dataframe=true)

performance_df

1: ∑p(o) = 1.0
2: ∑p(a) =  1.0


,I_ow,I_ao,I_awgo,H_o,H_a,H_ogw,H_ago,H_agow,E_U,Objective_value
1,-3.304634722310619e-17,1.5129570454871646e-17,0.8230503015899628,4.155149016627534,2.246499179958973,4.155149016627534,2.246499179958973,1.4234488783690102,1.726004705136832,0.9029544035468691
